In [1]:
import findspark
findspark.init('/Users/lmh/Programs/spark-2.2.0-bin-hadoop2.7')
from pyspark.sql import SparkSession 
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from matplotlib import pyplot as plt
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
rdd = sc.textFile('./data/ratings_Sports_and_Outdoors.csv')
rdd = rdd.map(lambda x: x.split(','))
rdd.take(10)

[[u'A3PMSRCL80KSA1', u'0000031852', u'4.0', u'1388275200'],
 [u'A1SNLWGLFXD70K', u'0000031852', u'4.0', u'1392940800'],
 [u'A1KJ4CVG87QW09', u'0000031852', u'4.0', u'1389657600'],
 [u'AA9ITO6ZLZW6', u'0000031852', u'5.0', u'1399507200'],
 [u'APJ5ULJ1RMZ4', u'0000031852', u'1.0', u'1398556800'],
 [u'A2PAVURT4NOHE1', u'0000031852', u'5.0', u'1388361600'],
 [u'A3URQ0LXLV46E9', u'0000031852', u'4.0', u'1400544000'],
 [u'A2681T699HV6H1', u'0000031895', u'4.0', u'1384905600'],
 [u'A2EPSZKEG06QZE', u'0000031895', u'2.0', u'1396224000'],
 [u'A23K73OVXJ04EG', u'0000031895', u'5.0', u'1391212800']]

In [3]:
user = rdd.map(lambda x: x[0])
item = rdd.map(lambda x: x[1])

In [4]:
def mergedic(x, y):
    x.update(y)
    return x

In [5]:
userd = user.distinct().zipWithIndex()
userd = userd.map(lambda x: {x[0]: x[1] + 1})
user_dic = userd.reduce(mergedic)
userd.take(10)

[{u'A3MW97MRTGPCAP': 1},
 {u'A3PPRXETCI2HX5': 2},
 {u'A3L480V4GK0B0T': 3},
 {u'A148SVSWKTJKU6': 4},
 {u'A2RRF4ZGRCWJWG': 5},
 {u'A3JRLXWMDGX4VW': 6},
 {u'ADKWDBXI65X2P': 7},
 {u'A2U38WAQPVD4XL': 8},
 {u'A2F2ZBEEYXXTHV': 9},
 {u'AXFFNC96ND85Y': 10}]

In [6]:
user_dic

{u'A3PPRXETCI2HX5': 2,
 u'A26PLO1YEGTPP9': 498138,
 u'A3RH35IIFVKCLW': 1493991,
 u'A2U38WAQPVD4XL': 8,
 u'A2QUIAST95HB0H': 997145,
 u'A148SVSWKTJKU6': 4,
 u'AVZ8TIB0FZ3FA': 498142,
 u'A164G1VJ12S9SO': 498177,
 u'A1IB9NI163FJFB': 1593480,
 u'A1G0UIQT2SYA0A': 997153,
 u'A2PQOEDRTY3V9D': 659172,
 u'A3MBAH3KEAON2U': 277830,
 u'ACMXKYK1R1LLZ': 1494058,
 u'A2KDDPJUNWC5CA': 11,
 u'AKU4ND0CF5Q0Z': 498154,
 u'A2UWPUZPSE4N8X': 997157,
 u'A616KZ1GNMIPH': 14,
 u'A1T8OKZED6537X': 997158,
 u'A8IFPO20VHIE1': 498156,
 u'A2W8QQCJ6O45C4': 997160,
 u'ADS7DIIOSIZ80': 1742638,
 u'A3PG322CVH40G': 1494114,
 u'A262HOVE07UNTU': 59965,
 u'A35FK4UCXR5KJB': 368683,
 u'A1GJ0J2LGQAGQB': 997163,
 u'A1A4RZ8CXVVAYQ': 1494011,
 u'AWJMEOZMWOC6': 498284,
 u'AS18XF83P0WA9': 1323923,
 u'A2R4BU3S05BGQU': 997176,
 u'A33CFTVK5LZVT8': 262332,
 u'ARYDFKRGOOFJA': 33,
 u'A3G2V3YKW7OJOC': 1494218,
 u'A10CTALTBBNUD9': 2325,
 u'A329DD1AK5RCR5': 498345,
 u'A2ESCDNNXE2BFB': 1743196,
 u'AEY6Y3C9JCEEA': 36,
 u'A2V499WHBFNJ8': 498170,
 u

In [7]:
itemd = item.distinct().zipWithIndex()
itemd = itemd.map(lambda x: {x[0]: x[1] + 1})
item_dic = itemd.reduce(mergedic)

In [8]:
cb = rdd.map(lambda (user, item, rating, timestamp): [user, item, rating, user_dic[user], item_dic[item]])

In [9]:
cbrdd = cb.map(lambda x: Row(username = x[0],
                             itemname = x[1],
                             rating = float(x[2]),
                             userid = int(x[3]),
                             itemid = int(x[4])))

In [10]:
df = spark.createDataFrame(cbrdd)

In [11]:
tdf = df.groupby('userid').agg({'rating': 'mean'})
df = tdf.join(df, tdf.userid == df.userid).drop(tdf.userid)
df = df.withColumnRenamed('avg(rating)', 'usermean')

In [12]:
tdf = df.groupby('itemid').agg({'rating': 'mean'})
df = tdf.join(df, tdf.itemid == df.itemid).drop(tdf.itemid)
df = df.withColumnRenamed('avg(rating)', 'itemmean')

In [13]:
df.repartition(1).write.csv('tempcsv', sep = ',')